In [1]:
from utils.trainer import Trainer

### set training parameters

In [2]:
MODEL_TYPE = "VGG16"
DATASET_TYPE = "MNIST_SPLIT"
TRAINING_MODE = "nn"
local = True

### initialize Trainer Instance

In [7]:
trainer = Trainer(model_type=MODEL_TYPE, dataset=DATASET_TYPE, mode=TRAINING_MODE, local=True)

{'mainDS': 'mnist', 'gray': True, 'torch': True, 'num_classes': 10, 'local': True}
load dataset from directory
load grayscale dataset 




/home/bot/anaconda3/envs/torch-gpu/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bot/anaconda3/envs/torch-gpu/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


### trigger training

In [8]:
trainer.train()

Training-Process started!
Epoch:0	Time:31.63	Train Loss:0.18	Train Acc:94.57	Test Acc98.33
Epoch:1	Time:31.53	Train Loss:0.04	Train Acc:98.97	Test Acc98.37
Epoch:2	Time:31.77	Train Loss:0.02	Train Acc:99.56	Test Acc98.79
Epoch:3	Time:31.78	Train Loss:0.01	Train Acc:99.92	Test Acc98.98
Epoch:4	Time:31.77	Train Loss:0.00	Train Acc:99.98	Test Acc99.02
Epoch:5	Time:31.86	Train Loss:0.00	Train Acc:99.99	Test Acc98.98
Epoch:6	Time:31.84	Train Loss:0.00	Train Acc:99.99	Test Acc99.03
Epoch:7	Time:31.67	Train Loss:0.00	Train Acc:99.99	Test Acc99.03
Epoch:8	Time:31.67	Train Loss:0.00	Train Acc:99.99	Test Acc99.05
Epoch:9	Time:31.67	Train Loss:0.00	Train Acc:100.00	Test Acc99.08


In [4]:
model = trainer.model
print(model)

DataParallel(
  (module): VGG16(
    (feature): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
      (13): MaxPool2d(kernel_size=2, stride=2, padding=

In [20]:
import torch
import torch.utils.data as data
from torchvision import transforms, datasets
import os 

val_indices = torch.load("/home/bot/coding/bachelorarbeit/ba_code/reinforcment_based/data/MNIST_SPLIT/val_indices.pth")
valloader = data.DataLoader(datasets.MNIST(
            "data/mnist", transform=transforms.Compose(
                            [
                                transforms.Grayscale(num_output_channels=3),
                                transforms.ToTensor(),
                                transforms.Resize((64, 64)),
                                transforms.Normalize(
                                    mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]
                                ),
                            ]
                        ), target_transform=None, download=True
        ), batch_size=32, sampler=data.SubsetRandomSampler(val_indices))

In [21]:
loss, cnt, ACC = 0.0, 0, 0

for img, iden in valloader:
    img, iden = img.to("cuda"), iden.to("cuda")
    bs = img.size(0)
    iden = iden.view(-1)

    out_prob = model(img)[-1]
    out_iden = torch.argmax(out_prob, dim=1).view(-1)
    ACC += torch.sum(iden == out_iden).item()
    cnt += bs

In [22]:
print(ACC*100 / cnt)

99.06666666666666
